# Chapter 4: Data Manipulation and Analysis Using Pandas

This chapter teaches you how to load, inspect, clean, transform, and analyze data using **Pandas** — with hands-on examples, exercises, and a mini-project.

## Introduction

**Why Pandas is Essential for Data Analysts**

In data analytics, most of your time is spent on things like:
- 📥 Importing data (CSV, Excel, databases)
- 🔍 Understanding what the data looks like
- 🧹 Cleaning issues (missing values, wrong types, duplicates)
- 🔄 Transforming it into a useful shape
- 📊 Summarizing it (grouping, aggregating)

Pandas is the most common Python library for these jobs — and mastering it will make you productive in nearly any data analytics role.

**What you'll learn in this chapter:**
- How to load data from files and databases
- How to inspect, clean, and transform data
- How to filter, sort, and aggregate data
- How to merge multiple datasets
- How to work with dates and time series
- How to export your results

**What you'll build:** A complete workflow that goes from "raw data" → "clean data" → "summary + chart" → "exported results".

**Prerequisites:** Basic Python knowledge (Chapter 2) and familiarity with NumPy arrays (Chapter 3) will help, but are not strictly required.

## Setup

If you haven’t installed Pandas yet, you can install it in your environment with:

- `pip install pandas`

We’ll also use NumPy (for numeric helpers) and Matplotlib (for a few quick plots).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.__version__

## 1) Pandas architecture: `Series` and `DataFrame`

Think of Pandas like this:
- A **Series** is a *single column* (1D) with an **index** (labels).
- A **DataFrame** is a *table of columns* (2D). Every column is a Series, and rows are aligned by an index.

Why this matters:
- Most analysis is “column-based”: you compute new columns, filter rows, group, and summarize.
- The **index** helps Pandas align data safely during operations (especially when merging or adding data).

In [ ]:
s = pd.Series([10, 20, 30], index=["a", "b", "c"])
df = pd.DataFrame({
    "product": ["Pen", "Notebook", "Pencil"],
    "price": [1.50, 3.00, 0.75],
    "in_stock": [True, True, False]
})

s, df

> **Tip (Beginner-friendly rule):** When you see “index” in Pandas, think “row labels”.
>
> By default, Pandas uses `0, 1, 2, ...` but you can use something meaningful (like an `order_id`).

## 2) Importing data from files and databases

In the real world, you often start by importing data from:
- CSV files (`pd.read_csv`)
- Excel files (`pd.read_excel`)
- JSON (`pd.read_json`)
- databases (e.g., SQLite) using `pd.read_sql_query`
- built-in datasets from libraries like **seaborn** using `sns.load_dataset`

We'll use the famous **tips** dataset from seaborn — a real-world dataset of restaurant tips that's perfect for learning Pandas.

In [ ]:
import seaborn as sns

# Load the tips dataset from seaborn (no download needed - it's built-in)
# This dataset contains information about restaurant bills and tips
tips = sns.load_dataset("tips")

# For consistency with our examples, we'll rename and select columns
orders = tips.rename(columns={
    "total_bill": "unit_price",
    "size": "quantity"
}).copy()

# Add order_id and order_date for our exercises
orders["order_id"] = range(1001, 1001 + len(orders))
orders["order_date"] = pd.date_range(start="2025-01-01", periods=len(orders), freq="h")
orders["customer"] = orders["sex"].map({"Male": "Alex", "Female": "Jordan"}) + "_" + orders.index.astype(str)
orders["product"] = orders["time"].map({"Lunch": "Lunch Special", "Dinner": "Dinner Menu"})

orders = orders[["order_id", "customer", "product", "quantity", "unit_price", "order_date", "tip", "day", "smoker"]].head(20)
orders

In [ ]:
import sqlite3

conn = sqlite3.connect(":memory:")

orders.to_sql("orders", conn, index=False, if_exists="replace")

query = """
SELECT customer, SUM(quantity * unit_price) AS revenue
FROM orders
GROUP BY customer
ORDER BY revenue DESC
LIMIT 10
"""

revenue_by_customer = pd.read_sql_query(query, conn)
revenue_by_customer

> **Common mistake:** letting Pandas guess the wrong types.
>
> For example, dates usually load as plain text first. We’ll fix that using `pd.to_datetime` later.

## 3) Data inspection and summary methods

Before you clean or analyze, do a quick inspection. Your goals are:
- see a few rows
- check column names
- check data types
- look for missing values
- get quick summary statistics

In [ ]:
orders.head()

In [ ]:
orders.info()

In [ ]:
orders.describe(include="all")

In [ ]:
orders.isna().sum()

### Exercise 1 — Quick inspection
1. How many rows and columns are in `orders`?
2. What are the numeric columns?
3. Which customers appear in the data, and how many orders does each have?

*Try it yourself first, then run the solution cell.*

In [ ]:
print("Shape (rows, cols):", orders.shape)

numeric_cols = orders.select_dtypes(include="number").columns.tolist()
print("Numeric columns:", numeric_cols)

orders_per_customer = orders["customer"].value_counts()
orders_per_customer

## 4) Indexing and selection techniques

Selecting the right rows/columns is a daily task. The two most important tools are:
- `loc` for **label-based** selection
- `iloc` for **position-based** selection

You’ll also use boolean filtering (conditions) a lot.

> **Warning (very common bug):** avoid chained indexing like `df[df['x'] > 0]['y'] = 1`.
> It can silently fail. Prefer `df.loc[mask, 'y'] = 1`.

In [ ]:
# Column selection
orders[["order_id", "customer", "product"]]

In [ ]:
# Boolean filtering (rows) - filter for Dinner Menu with party size >= 3
mask = (orders["product"] == "Dinner Menu") & (orders["quantity"] >= 3)
orders.loc[mask, ["order_id", "customer", "product", "quantity"]]

### Exercise 2 — Filtering practice
Create a filtered DataFrame containing only orders where `smoker` is "Yes", and only show the columns `product`, `quantity`, and `unit_price`.

*Try first, then run the solution.*

In [ ]:
# Solution: filter for smokers
smoker_orders = orders.loc[orders["smoker"] == "Yes", ["product", "quantity", "unit_price"]]
smoker_orders

## 5) Sorting and filtering data

Once you can select rows, you’ll often want to:
- sort rows (e.g., biggest sales first)
- filter using helper methods like `isin`, `between`, or `query`

In [ ]:
orders2 = orders.copy()
orders2["total"] = orders2["quantity"] * orders2["unit_price"]

orders2.sort_values(by="total", ascending=False)

In [ ]:
# Keep only Lunch Special or Dinner Menu products
orders2[orders2["product"].isin(["Lunch Special", "Dinner Menu"])]

## 6) Handling missing and duplicate data

Real datasets often contain:
- missing values (`NaN`)
- duplicate rows

A good cleaning approach is:
1. *measure* the problem (how many missing/duplicates?)
2. decide a strategy (drop, fill, or fix)
3. apply the strategy and re-check

In [ ]:
dirty = orders2.copy()
dirty.loc[1, "unit_price"] = np.nan  # introduce missing value
dirty = pd.concat([dirty, dirty.iloc[[2]]], ignore_index=True)  # duplicate a row
dirty

In [ ]:
print("Missing values per column:")
print(dirty.isna().sum())

print("\nDuplicate rows (count):", dirty.duplicated().sum())

In [ ]:
# Strategy example: fill missing unit_price using the median price
cleaned = dirty.copy()
median_price = cleaned["unit_price"].median()
cleaned["unit_price"] = cleaned["unit_price"].fillna(median_price)

# Remove exact duplicate rows
cleaned = cleaned.drop_duplicates()

cleaned.isna().sum(), cleaned.duplicated().sum()

> **Tip:** Don’t automatically drop rows with missing values.
>
> Ask: *Is missingness meaningful?* For example, missing prices might mean “price not recorded”, which could be a data quality problem you need to report.

## 7) Data type conversion

Data types matter because they control what you can do:
- numeric columns can be summed/averaged
- datetime columns can be sorted, resampled, and grouped by time

Two very common conversions:
- `pd.to_numeric(...)`
- `pd.to_datetime(...)`

In [ ]:
typed = cleaned.copy()
typed["order_date"] = pd.to_datetime(typed["order_date"], errors="coerce")

typed.dtypes

> **Warning:** `errors='coerce'` turns invalid values into `NaT` (missing datetime).
>
> That's often useful, but it can also hide a data quality problem. Always check how many values became missing after conversion.

### Exercise 4 — Data type conversion
The `orders` DataFrame has `order_date` as a string. Convert it to datetime and extract the day of the week (Monday=0, Sunday=6) into a new column called `weekday`.

*Try first, then run the solution.*

In [ ]:
# Solution for Exercise 4
orders_with_date = orders.copy()
orders_with_date["order_date"] = pd.to_datetime(orders_with_date["order_date"], errors="coerce")
orders_with_date["weekday"] = orders_with_date["order_date"].dt.dayofweek
orders_with_date[["order_id", "order_date", "weekday"]]

In [ ]:
typed["order_date"].isna().sum()

## 8) GroupBy and aggregation

`groupby` answers questions like:
- total revenue per customer
- average quantity per product
- number of orders per day

A typical pattern is:
1. create a useful measure (like `total = quantity * unit_price`)
2. group by one or more columns
3. aggregate (sum, mean, count, etc.)

In [ ]:
analysis = typed.copy()
analysis["total"] = analysis["quantity"] * analysis["unit_price"]

by_customer = (
    analysis.groupby("customer", as_index=False)
    .agg(orders=("order_id", "count"), revenue=("total", "sum"), avg_order_value=("total", "mean"))
    .sort_values(by="revenue", ascending=False)
)

by_customer

In [ ]:
# Visual example: revenue per customer
ax = by_customer.set_index("customer")["revenue"].plot(kind="bar", title="Revenue by Customer")
ax.set_xlabel("Customer")
ax.set_ylabel("Revenue")
plt.show()

### Exercise 3 — GroupBy
Compute total quantity sold per product and sort from highest to lowest.

*Try first, then run the solution.*

In [ ]:
qty_by_product = (
    analysis.groupby("product", as_index=False)
    .agg(total_qty=("quantity", "sum"))
    .sort_values(by="total_qty", ascending=False)
)
qty_by_product

## 9) Merging, joining, and concatenation

Data often comes in multiple tables. You combine them using:
- `pd.merge(...)` (SQL-style joins: inner, left, right, outer)
- `df.join(...)` (join by index, or by a key column)
- `pd.concat([...])` (stack tables vertically or place them side-by-side)

We’ll create a small product table and join it to orders.

In [ ]:
# Create a product category table to join with our orders
products = pd.DataFrame({
    "product": ["Lunch Special", "Dinner Menu"],
    "category": ["Daytime", "Evening"],
    "avg_prep_time_mins": [15, 25]
})

merged = pd.merge(analysis, products, on="product", how="left")
merged.head()

In [ ]:
# Concatenation example (stack rows)
more_orders = pd.DataFrame({
    "order_id": [2001],
    "customer": ["Dana_999"],
    "product": ["Lunch Special"],
    "quantity": [4],
    "unit_price": [18.50],
    "order_date": [pd.Timestamp("2025-01-15")],
    "total": [74.00],
})

combined = pd.concat([analysis, more_orders], ignore_index=True)
combined.tail()

> **Common mistakes when merging:**
> - joining on columns with different spelling/case (`Product` vs `product`)
> - joining on columns with different types (string vs integer)
> - accidental "many-to-many" joins that duplicate rows
>
> Tip: `validate='many_to_one'` (as shown) can catch mistakes early.

### Exercise 5 — Merging practice
Create a `customers` DataFrame with columns `customer` and `city` (make up 3-4 customers). Then merge it with the `analysis` DataFrame to add the city information to each order.

*Try first, then run the solution.*

In [ ]:
# Solution for Exercise 5 - Get unique customers first, then create city mapping
unique_customers = analysis["customer"].unique()[:4]
customers = pd.DataFrame({
    "customer": unique_customers,
    "city": ["New York", "Los Angeles", "Chicago", "Houston"][:len(unique_customers)]
})

orders_with_city = pd.merge(analysis, customers, on="customer", how="left")
orders_with_city[["order_id", "customer", "city", "product", "total"]].head()

## 10) Reshaping data: `pivot` and `melt`

Data can be **wide** or **long**:
- **wide:** many columns (e.g., one column per month)
- **long:** fewer columns, more rows (e.g., month stored as values)

Why reshape?
- many visualizations and groupby operations prefer long data
- reporting tables often prefer wide data

In [ ]:
daily_revenue = (
    analysis.groupby(["order_date", "product"], as_index=False)
    .agg(revenue=("total", "sum"))
)

wide = daily_revenue.pivot(index="order_date", columns="product", values="revenue").fillna(0)
wide

In [ ]:
long_again = wide.reset_index().melt(id_vars="order_date", var_name="product", value_name="revenue")
long_again.head()

> **Tip:** Use `pivot` when you want to reshape from long to wide format (e.g., for reporting tables).
> Use `melt` when you want to go from wide to long format (e.g., for plotting or groupby operations).

### Exercise 6 — Reshaping practice
Using the `wide` DataFrame created above, convert it back to long format using `melt`, then filter to show only rows where revenue > 0.

*Try first, then run the solution.*

In [ ]:
# Solution for Exercise 6
long_filtered = wide.reset_index().melt(id_vars="order_date", var_name="product", value_name="revenue")
long_filtered = long_filtered[long_filtered["revenue"] > 0]
long_filtered

## 11) Working with dates and time series

When a column is a datetime type, Pandas gives you powerful tools:
- `.dt` accessor (year, month, day, weekday, etc.)
- sorting by time
- resampling (daily → weekly/monthly)
- rolling averages

We’ll build a simple daily revenue time series and plot it.

In [ ]:
ts = (
    analysis.groupby("order_date", as_index=True)["total"]
    .sum()
    .sort_index()
)

ts

In [ ]:
ax = ts.plot(marker="o", title="Daily Revenue")
ax.set_xlabel("Date")
ax.set_ylabel("Revenue")
plt.show()

In [ ]:
# Rolling average smooths short-term noise (window=2 days here)
rolling = ts.rolling(window=2, min_periods=1).mean()

ax = ts.plot(marker="o", label="Daily")
rolling.plot(ax=ax, label="2-day rolling mean")
ax.legend()
ax.set_title("Daily Revenue with Rolling Mean")
plt.show()

> **Tip:** Rolling averages are useful for smoothing noisy data and identifying trends.
> Common window sizes: 7 days (weekly), 30 days (monthly), depending on your data frequency.

### Exercise 7 — Time series practice
Using the `ts` time series created above, compute a cumulative sum of daily revenue and display it.

*Try first, then run the solution.*

In [ ]:
# Solution for Exercise 7
cumulative_revenue = ts.cumsum()

ax = cumulative_revenue.plot(marker="o", title="Cumulative Daily Revenue")
ax.set_xlabel("Date")
ax.set_ylabel("Cumulative Revenue")
plt.show()

cumulative_revenue

## 12) Exporting processed data

After cleaning/analysis, you often export results for:
- reporting
- dashboards
- sharing with other teams

Common exports:
- CSV (`to_csv`)
- Excel (`to_excel`)
- Parquet (`to_parquet`) for efficient analytics (optional)

In [ ]:
# Export a summary table to CSV (writes to the current working directory)
output_path = "chapter04_revenue_by_customer.csv"
by_customer.to_csv(output_path, index=False)

output_path

> **Warning:** Exporting overwrites files by default if the name already exists.
>
> If you're experimenting, use unique filenames or export into a dedicated folder.

> **Tip:** For large datasets, consider using `to_parquet()` format instead of CSV. Parquet files are:
> - Much smaller (compressed)
> - Faster to read/write
> - Preserve data types automatically

## Mini‑Project: Clean and analyze a real-world dataset

In this mini‑project you will:
1. Load the seaborn **flights** dataset (monthly airline passengers)
2. Clean it (types, missing data check)
3. Compute aggregations and summarize by year and month
4. Plot a simple time trend
5. Export the cleaned dataset and a summary table

This is a realistic end‑to‑end workflow for beginner analysts using a real dataset.

In [ ]:
# Load the flights dataset from seaborn
raw = sns.load_dataset("flights")

# Add some "messy" elements for cleaning practice
raw_messy = raw.copy()
raw_messy.loc[5, "passengers"] = np.nan  # introduce missing value
raw_messy = pd.concat([raw_messy, raw_messy.iloc[[10]]], ignore_index=True)  # duplicate row

raw_messy.head(15)

In [ ]:
clean = raw_messy.copy()

# 1) Check for missing values
print("Missing values before cleaning:")
print(clean.isna().sum())

# 2) Handle missing values - fill with median
clean["passengers"] = clean["passengers"].fillna(clean["passengers"].median())

# 3) Remove duplicates
clean = clean.drop_duplicates()

# 4) Create a proper date column for time series analysis
clean["date"] = pd.to_datetime(clean["year"].astype(str) + "-" + clean["month"].astype(str) + "-01")

print("\nMissing values after cleaning:")
print(clean.isna().sum())
print(f"\nShape: {clean.shape}")
clean.head()

In [ ]:
# Summarize by year
summary_year = (
    clean.groupby("year", as_index=False)
    .agg(total_passengers=("passengers", "sum"), avg_passengers=("passengers", "mean"))
    .sort_values(by="year")
)
summary_year

In [ ]:
# Time series plot of passengers
ts = clean.set_index("date")["passengers"].sort_index()

ax = ts.plot(figsize=(12, 4), title="Monthly Airline Passengers Over Time")
ax.set_xlabel("Date")
ax.set_ylabel("Number of Passengers (thousands)")
plt.tight_layout()
plt.show()

ts.head()

In [ ]:
clean.to_csv("chapter04_clean_flights.csv", index=False)
summary_year.to_csv("chapter04_flights_summary_by_year.csv", index=False)

["chapter04_clean_flights.csv", "chapter04_flights_summary_by_year.csv"]

## Optional resources
- Pandas User Guide: https://pandas.pydata.org/docs/user_guide/
- 10 minutes to pandas: https://pandas.pydata.org/docs/user_guide/10min.html
- Pandas `merge` docs: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html
- Time series / resampling: https://pandas.pydata.org/docs/user_guide/timeseries.html

## Summary / Key Takeaways

**Core Concepts:**
- **Series** = one labeled column; **DataFrame** = a table of Series.
- The **index** provides row labels and enables powerful alignment during operations.

**Workflow Essentials:**
- Start every analysis by inspecting: `head`, `info`, `describe`, missing counts.
- Use `loc`/`iloc` + boolean masks for safe, readable selection.
- Clean data intentionally: measure issues → choose a strategy → re-check.
- Convert types early (especially dates) using `pd.to_datetime` / `pd.to_numeric`.

**Analysis & Aggregation:**
- Summarize with `groupby` + `agg`, and visualize quick insights with simple plots.
- Combine tables with `merge`/`join`, stack with `concat`, reshape with `pivot`/`melt`.

**Time Series & Export:**
- Use `.dt` accessor for datetime operations; use `resample` for time-based aggregation.
- Export results with `to_csv` (and other formats) to share or report.

**Exercises Completed:**
1. Quick inspection (shape, dtypes, value counts)
2. Filtering practice (boolean masks with `loc`)
3. GroupBy aggregation
4. Data type conversion (datetime, weekday extraction)
5. Merging DataFrames
6. Reshaping with pivot/melt
7. Time series cumulative sum

**What's Next:** In Chapter 5, you'll learn how to create effective visualizations to communicate your data insights using Matplotlib and Seaborn.